In [1]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, CLIPTextModel
import torch
from PIL import Image
import json

ModuleNotFoundError: No module named 'transformers'

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def predict_step(image_paths, gen_kwargs):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [2]:
with open('./Meta-DETR/data/voc_fewshot_split1/seed1/3shot.json') as f:
    data_3shot = json.load(f)

NameError: name 'json' is not defined

In [ ]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
desc = predict_step(images)

In [ ]:
clip_embedding_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
embedding_tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
inputs = tokenizer(desc, padding=True, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output 